## LightGBM-Optunaのサンプルコード

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# LightGBM
import optuna.integration.lightgbm as lgb

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [4]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

# trainのデータセットの2割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [5]:
# ベストなパラメータ、途中経過を保存する
params = {
    "objective": "mean_squared_error",
    "metric": "rmse",
    "verbosity": 10,
    "boosting_type": "gbdt",
    "random_seed": 0,
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_eval],
    early_stopping_rounds=100,
    verbose_eval=50,
)

[I 2021-05-28 14:55:03,539] A new study created in memory with name: no-name-45e68d47-206d-4cf1-a90a-a1f35b98fdc5
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 2.29947	valid_1's rmse: 2.96402
[100]	valid_0's rmse: 1.58202	valid_1's rmse: 2.96425


feature_fraction, val_score: 2.924797:  14%|#4        | 1/7 [00:00<00:04,  1.40it/s]

Early stopping, best iteration is:
[38]	valid_0's rmse: 2.59897	valid_1's rmse: 2.9248
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 2.31706	valid_1's rmse: 3.18378
[100]	valid_0's rmse: 1.53914	valid_1's rmse: 2.98479
[150]	valid_0's rmse: 1.16032	valid_1's rmse: 2.96396
[200]	valid_0's rmse: 0.913527	valid_1's rmse: 2.95325
[250]	valid_0's rmse: 0.750357	valid_1's rmse: 2.93815
[300]	valid_0's rmse: 0.627879	valid_1's rmse: 2.92698
[350]	valid_0's rmse: 0.53372	valid_1's rmse: 2.92501
[400]	valid_0's rmse: 0.460032	valid_1's rmse: 2.90336
[450]	valid_0's rmse: 0.400247	valid_1's rmse: 2.89365
[500]	valid_0's rmse: 0.354627	valid_1's rmse: 2.88808
[550]	valid_0's rmse: 0.310126	valid_1's rmse: 2.88638
[600]	valid_0's rmse: 0.278825	valid_1's rmse: 2.89207


feature_fraction, val_score: 2.885625:  29%|##8       | 2/7 [00:04<00:07,  1.57s/it]

...

Early stopping, best iteration is:
[32]	valid_0's rmse: 3.11094	valid_1's rmse: 3.21702
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 1.50814	valid_1's rmse: 2.88887
[100]	valid_0's rmse: 0.861931	valid_1's rmse: 2.83407
[150]	valid_0's rmse: 0.610213	valid_1's rmse: 2.84094
[200]	valid_0's rmse: 0.43464	valid_1's rmse: 2.82284
[250]	valid_0's rmse: 0.32357	valid_1's rmse: 2.82252
[300]	valid_0's rmse: 0.242245	valid_1's rmse: 2.82248


min_data_in_leaf, val_score: 2.747222:  80%|########  | 4/5 [00:08<00:02,  2.40s/it]

[350]	valid_0's rmse: 0.187453	valid_1's rmse: 2.82552
Early stopping, best iteration is:
[251]	valid_0's rmse: 0.320666	valid_1's rmse: 2.81985
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 0.736777	valid_1's rmse: 2.87848
[100]	valid_0's rmse: 0.336064	valid_1's rmse: 2.83597
[150]	valid_0's rmse: 0.205029	valid_1's rmse: 2.83596
[200]	valid_0's rmse: 0.127279	valid_1's rmse: 2.84236


min_data_in_leaf, val_score: 2.747222: 100%|##########| 5/5 [00:13<00:00,  2.80s/it]

Early stopping, best iteration is:
[134]	valid_0's rmse: 0.248272	valid_1's rmse: 2.83233


In [6]:
best_params = gbm.params
best_params

{'objective': 'mean_squared_error',
 'metric': 'rmse',
 'verbosity': 10,
 'boosting_type': 'gbdt',
 'random_seed': 0,
 'feature_pre_filter': False,
 'lambda_l1': 0.016677702506262326,
 'lambda_l2': 5.0208953125935345e-05,
 'num_leaves': 31,
 'feature_fraction': 0.7,
 'bagging_fraction': 0.9880321015000904,
 'bagging_freq': 4,
 'min_child_samples': 20}

In [7]:
# LightGBM推論
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

In [8]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [9]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.845409  2.744357  16.167263  4.020853
